<a href="https://colab.research.google.com/github/ogniandantchev/gap-map/blob/master/Europe_map_choropleth_untitled7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import folium
import geopandas as gpd
import pandas as pd
import json


In [4]:

europe_gdf= gpd.read_file("europe.geo.json")
# 1. Load GeoJSON data for Europe
# try:
#     europe_gdf = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
#     europe_gdf = europe_gdf[europe_gdf['continent'] == 'Europe']
# except Exception as e:
#     print(f"Error loading GeoJSON data: {e}. Please ensure geopandas is installed and data is available.")
#     exit()

# Filter out Russia and Belarus from GeoJSON data
#europe_countries_gdf = europe_gdf[~europe_gdf['name'].isin(['Russia', 'Belarus'])]

# List of European Countries (for GDP data filtering and validation later)
european_countries_list = europe_gdf['name'].tolist()

# 2. Load GDP data for European countries
# Using a simplified GDP data (replace with actual data loading and processing)
# **IMPORTANT**: Replace this with actual GDP data loading from a reliable source (e.g., World Bank, IMF)
# For demonstration, creating a dummy GDP DataFrame.
# In a real application, you would load from CSV/Excel, clean and preprocess.

# **Replace this with your actual GDP data loading and processing**
# Example of loading from CSV (you would need to download GDP data CSV)
# gdp_df = pd.read_csv('path_to_your_gdp_data.csv')
# gdp_df = gdp_df[['Country Name', 'GDP_Value_Column_Name']] # Select relevant columns
# gdp_df.columns = ['name', 'gdp'] # Rename columns for merging

# Dummy GDP Dataframe - Replace with actual data loading
data = {'name': ['Germany', 'France', 'United Kingdom', 'Italy', 'Spain', 'Poland', 'Netherlands', 'Belgium', 'Sweden', 'Austria', 'Romania', 'Czechia', 'Greece', 'Portugal', 'Hungary', 'Denmark', 'Finland', 'Slovakia', 'Ireland', 'Croatia', 'Bulgaria', 'Lithuania', 'Latvia', 'Slovenia', 'Estonia', 'Cyprus', 'Luxembourg', 'Malta', 'Iceland', 'Norway', 'Switzerland', 'Albania', 'Bosnia and Herzegovina', 'North Macedonia', 'Serbia', 'Montenegro', 'Kosovo'],
        'gdp': [4223, 2924, 3100, 2050, 1430, 680, 1020, 590, 630, 470, 300, 280, 220, 240, 180, 390, 290, 120, 530, 70, 90, 60, 40, 60, 30, 25, 80, 17, 28, 520, 810, 18, 20, 15, 62, 6, 9]} # GDP in Billions USD (Example values)
gdp_df = pd.DataFrame(data)



In [14]:
# Ensure GDP data is for countries in our European GeoJSON and list
gdp_df_europe = gdp_df[gdp_df['name'].isin(european_countries_list)]

# Validate if all countries in GDP are in GeoJSON (and vice versa for relevant countries)
countries_in_gdp = set(gdp_df_europe['name'])
countries_in_geojson = set(europe_gdf['name'])
missing_in_geojson = countries_in_gdp - countries_in_geojson
missing_in_gdp = countries_in_geojson - countries_in_gdp

if missing_in_geojson:
    print(f"Warning: Countries in GDP data but not in GeoJSON: {missing_in_geojson}. These will be excluded.")
if missing_in_gdp:
    print(f"Warning: Countries in GeoJSON but not in GDP data: {missing_in_gdp}. These will be excluded.")

# Merge GDP data into GeoJSON dataframe
merged_gdf = europe_gdf.merge(gdp_df_europe, on='name', how='inner')

# 3. Calculate total GDP and prorate AI Revenue
total_gdp = merged_gdf['gdp'].sum()
total_ai_revenue = 60 # Billion EUR

merged_gdf['ai_revenue_prorated'] = (merged_gdf['gdp'] / total_gdp) * total_ai_revenue

# Convert AI revenue to EUR (assuming GDP is in USD and conversion rate is needed)
# For simplicity, assuming 1 USD = 1 EUR for demonstration. **Adjust as needed based on actual data.**
merged_gdf['ai_revenue_prorated_eur'] = merged_gdf['ai_revenue_prorated'] # No conversion for now, assuming USD is close to EUR for demo.


# 4. Create Choropleth map
# Convert GeoDataFrame to GeoJSON for Folium
europe_geojson = json.loads(merged_gdf.to_json())


# Initialize Folium map centered on Europe
europe_map = folium.Map(location=[50, 10], zoom_start=3.5)

# Choropleth layer
# Create the choropleth
choropleth = folium.Choropleth(
    geo_data=europe_geojson,  # The GeoJSON data
    data=merged_gdf,          # The Pandas GeoDataFrame with your data
    columns=['name', 'ai_revenue_prorated_eur'],  # Columns to use
    key_on='feature.properties.name', # How to join the data to the GeoJSON
    fill_color='YlGnBu',       # Color scale
    fill_opacity=0.7,        # Fill opacity
    line_opacity=0.2,        # Border opacity
    legend_name='Prorated AI Revenue (Billion EUR)', # Legend title
    highlight=True,           # Enable highlighting on mouseover
    tooltip=folium.GeoJsonTooltip(fields=['name', 'ai_revenue_prorated_eur'], aliases=['Country: ', 'AI Revenue (Billion EUR): '])
).add_to(europe_map)

# 2. Create a GeoJson layer for the tooltips
geojson_layer = folium.GeoJson(
    europe_geojson,  # Use the same GeoJSON
    name="Countries",  # Give it a name (optional)
    tooltip=folium.GeoJsonTooltip( # Attach the tooltip here
        fields=['name', 'ai_revenue_prorated_eur'],
        aliases=['Country: ', 'AI Revenue (Billion EUR): '],
        labels=True,
        sticky=True,
    )
).add_to(europe_map) # Add the GeoJson layer to the map
# Add Layer Control
folium.LayerControl().add_to(europe_map)

# Save the map
europe_map.save("european_ai_revenue_map.html")

print("Choropleth map created and saved as european_ai_revenue_map.html")

Choropleth map created and saved as european_ai_revenue_map.html
